In [1]:
import graphlab as gl
import pickle
import pandas as pd
import numpy as np

A newer version of GraphLab Create (v2.0.1) is available! Your current version is v2.0.

You can use pip to upgrade the graphlab-create package. For more information see https://turi.com/products/create/upgrade.


In [2]:
data_items = pickle.load(open("/Users/marvinbertin/Github/family_style_chat_bot/data/user_by_cuisine_by_dish_ratings.pkl", 'rb'))
data_cuisine = pickle.load(open("/Users/marvinbertin/Github/family_style_chat_bot/data/user_by_cuisine_ratings.pkl", 'rb'))

In [3]:
df_cuisine = pd.DataFrame(data_cuisine)
df_cuisine.head()

,user_id,item_id,rating
0,Brian,China,2
1,David,China,0.5
3,Jon,China,4
4,Marvin,China,0.5
6,Anne,China,2.5


In [4]:
data_items.keys()

['Italy',
 'Mexico',
 'Indonesia',
 'India',
 'Singapore',
 'France',
 'Vietnam',
 'China',
 'Thailand',
 'Japan',
 'Pizza']

In [8]:
class group_recommender(object):
    def __init__(self, cuisine_sf, dict_cuisine_items):
        self.cuisine = gl.SFrame(cuisine_sf)
        self.cuisine_items = dict_cuisine_items
    
    def recommend(self, group_list):
        group_name = "_".join(group_list)
        
        sf_avg_user = self.cuisine.filter_by(group_list, "user_id") \
                                  .groupby(key_columns='item_id',
                                           operations={'rating': gl.aggregate.MEAN('rating')})  
        
        sf_avg_user.add_column(gl.SArray([group_name] * len(sf_avg_user)), "user_id")
        print sf_avg_user
        
        sf_new = self.cuisine.append(sf_avg_user)
        
        model = gl.recommender.create(sf_new, target='rating')
    
        results = model.recommend([group_name], exclude_known=False)
        print results
        result_cuisine = results["item_id"][0]
        
        sf_items = gl.SFrame(self.cuisine_items[result_cuisine])
        model_items = gl.recommender.create(sf_items, target='rating')
        
        results_items = model_items.recommend(group_list, exclude_known=False, k = 2)
        print results_items
        
        name_items = results_items[["user_id", "item_id"]].to_numpy().tolist()
    
        recommendations = []
        for name in group_list:
            recommendations.append((name,
                                    map(lambda (n, meal): meal,
                                        filter(lambda (n, meal): n == name , name_items))))
        return recommendations
        
#         return  name_items.groupby(key_columns= "user_id", \
#                                   operations = {"item_id":gl.aggregate.CONCAT("item_id")})

In [11]:
group_list = np.random.choice(df_cuisine["user_id"].unique(), size = 3, replace=False)
print group_list
model = group_recommender(df_cuisine, data_items)
result = model.recommend(group_list)

['Marvin' 'David' 'Brian']
+-----------+---------------+--------------------+
|  item_id  |     rating    |      user_id       |
+-----------+---------------+--------------------+
|   China   |      1.0      | Marvin_David_Brian |
|   Japan   |      3.25     | Marvin_David_Brian |
|   France  |      3.5      | Marvin_David_Brian |
|   Pizza   |      2.0      | Marvin_David_Brian |
| Indonesia |      4.75     | Marvin_David_Brian |
|  Vietnam  |      2.0      | Marvin_David_Brian |
|   Italy   |      4.0      | Marvin_David_Brian |
| Singapore |      3.25     | Marvin_David_Brian |
|   India   |      4.5      | Marvin_David_Brian |
|  Thailand | 2.83333333333 | Marvin_David_Brian |
+-----------+---------------+--------------------+
[10 rows x 3 columns]



Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 80 observations with 12 users and 11 items.

Data prepared in: 0.007234s

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.577251

Final training RMSE: 0.355506

+---------+----------------------------+---------------+------+
| user_id |          item_id           |     score     | rank |
+---------+----------------------------+---------------+------+
|  Marvin |   Indonesian Spiced Rice   | 4.95127811886 |  1   |
|  Marvin |  Indonesian Fried Noodles  | 4.52071991421 |  2   |
|  David  |      Indonesia Satay       | 3.96709844328 |  1   |
|  David  | Indonesian Banana Fritters | 3.46769091345 |  2   |
|  Brian  |  Indonesian Beef Rendang   | 4.89794454582 |  1   |
|  Brian  |  Indonesian Fried Noodles  | 2.99497542388 |  2   |
+---------+----------------------------+---------------+------+
[6 rows x 4 columns]



In [12]:
result

[('Marvin', ['Indonesian Spiced Rice', 'Indonesian Fried Noodles']),
 ('David', ['Indonesia Satay', 'Indonesian Banana Fritters']),
 ('Brian', ['Indonesian Beef Rendang', 'Indonesian Fried Noodles'])]